In [71]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import copy
np.random.seed(0)

# Data preprocess

In [2]:
drug8 = pd.read_csv('drug8.csv')
drug3 = pd.read_csv('drug3.csv')
drug10 = pd.read_csv('drug10.csv')
d1 = pd.read_csv('disease1.csv')
d2 = pd.read_csv('disease2.csv')
d5 = pd.read_csv('disease5.csv')

In [3]:
drug3_p = set(drug3['patid'].values)
drug10_p = set(drug10['patid'].values)
d2_p = set(d2['patid'].values)
d5_p = set(d5['patid'].values)

In [4]:
meta = pd.concat([drug8,drug3,drug10,d1,d2,d5]).drop_duplicates('patid')[['patid','GDR_CD','yrdob']]

C:\Users\peter\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
last_date = max(d1['lst_dt'])
d1 = d1.drop_duplicates('patid', keep='first')
d2 = d2.drop_duplicates('patid', keep='first')
d5 = d5.drop_duplicates('patid', keep='first')
drug8 = drug8[drug8['VXX_dt']<=last_date-60]
drug8_p = set(drug8['patid'].values)
d1_p = set(d1['patid'].values)

# Generate new records

In [6]:
patient = set(drug8['patid'].values)|\
          set(drug3['patid'].values)|\
          set(drug10['patid'].values)|\
          set(d2['patid'].values)|\
          set(d5['patid'].values)-\
          set(d1.loc[d1['fst_dt']<60,'patid'].values)

In [7]:
red = {'patid':[], 
       'date' : [],
       'd1_record': [],
       'drug8_record':[]
      }
for i in patient:
    red['patid'].append(i)
    if i in drug8_p: #If take the drug
        if i in d1_p: #If have the disease 1
            diagnose =  d1.loc[d1['patid']==i,'fst_dt'].values[0]
            sub = drug8[(drug8['patid']==i) & (drug8['VXX_dt']<diagnose-14) &  (drug8['sup_dt']>diagnose-60)]
            if len(sub) == 0: #if no record satisfies the time condition
                red['date'].append(np.random.choice(drug8[drug8['patid']==i]['VXX_dt'].values))
                red['d1_record'].append(0) #randomly pick one record
            else: #if at least one record satisfies the time condition
                red['date'].append(np.random.choice(sub['VXX_dt'].values))
                red['d1_record'].append(1) #randomly pick one record satisfying the time condition
        else: #If no disease 1
            red['date'].append(np.random.choice(drug8[drug8['patid']==i]['VXX_dt'].values))
            red['d1_record'].append(0) # randomly pick one record
        red['drug8_record'].append(1) 
        
    else: #If not take the drug
        if i in d1_p: #If have the disease 1
            red['date'].append(d1.loc[d1['patid']==i, 'fst_dt'].values[0] - np.random.randint(14,60))
            red['d1_record'].append(1) #randomly pick a date 14-60 days between getting the disease
        else: #If no disease 1
            red['date'].append(np.random.randint(500,2000))
            red['d1_record'].append(0) #randomly pick a date between 500 and 2000
        red['drug8_record'].append(0)
red = pd.DataFrame(red)    

In [9]:
red_tmp = pd.merge(red, drug3, on = 'patid', how = 'left')
red_tmp.loc[red_tmp['VXX_dt']<red_tmp['date'],'drug3']=1 #If taking drug3 before the date, set column ‘drug3’ to be 1
red_tmp.loc[red_tmp['drug3']!=1,'drug3']=0
red_tmp_10 = red_tmp.drop_duplicates('patid')
red_tmp_10=red_tmp_10[['patid','drug3']]
red_tmp_date = red_tmp[red_tmp['drug3'] == 1]
red_tmp_date.loc[:,'drug3_date'] = red_tmp_date['date'] - red_tmp_date['sup_dt'] 
red_tmp_date.loc[red_tmp_date['drug3_date']<0,'drug3_date'] = 0 #exacting all record with drug3 = 1, calculate the latest time taking drug 3
red_tmp_date = red_tmp_date.drop_duplicates('patid', keep='last')
red_tmp_date = red_tmp_date[['patid','drug3_date']]
red = pd.merge(red, red_tmp_10, on='patid', how ='left')
red = pd.merge(red, red_tmp_date, on = 'patid', how = 'left') #merge

red_tmp = pd.merge(red, drug10, on = 'patid', how = 'left')
red_tmp.loc[red_tmp['VXX_dt']<red_tmp['date'],'drug10']=1
red_tmp.loc[red_tmp['drug10']!=1,'drug10']=0
red_tmp_10 = red_tmp.drop_duplicates('patid')
red_tmp_10=red_tmp_10[['patid','drug10']]
red_tmp_date = red_tmp[red_tmp['drug10'] == 1]
red_tmp_date.loc[:,'drug10_date'] = red_tmp_date['date'] - red_tmp_date['sup_dt'] 
red_tmp_date.loc[red_tmp_date['drug10_date']<0,'drug10_date'] = 0
red_tmp_date = red_tmp_date.drop_duplicates('patid', keep='last')
red_tmp_date = red_tmp_date[['patid','drug10_date']]
red = pd.merge(red, red_tmp_10, on='patid', how ='left')
red = pd.merge(red, red_tmp_date, on = 'patid', how = 'left')

red_tmp = pd.merge(red, drug8, on = 'patid', how = 'left')
red_tmp.loc[red_tmp['VXX_dt']<red_tmp['date'],'drug8']=1
red_tmp.loc[red_tmp['drug8']!=1,'drug8']=0
red_tmp_10 = red_tmp.drop_duplicates('patid')
red_tmp_10=red_tmp_10[['patid','drug8']]
red_tmp_date = red_tmp[red_tmp['drug8'] == 1]
red_tmp_date.loc[:,'drug8_date'] = red_tmp_date['date'] - red_tmp_date['sup_dt'] 
red_tmp_date.loc[red_tmp_date['drug8_date']<0,'drug8_date'] = 0
red_tmp_date = red_tmp_date.drop_duplicates('patid', keep='last')
red_tmp_date = red_tmp_date[['patid','drug8_date']]
red = pd.merge(red, red_tmp_10, on='patid', how ='left')
red = pd.merge(red, red_tmp_date, on = 'patid', how = 'left')

red_tmp = pd.merge(red, d2, on = 'patid', how = 'left')
red_tmp.loc[red_tmp['fst_dt']<red_tmp['date'],'d2']=1
red_tmp.loc[red_tmp['d2']!=1,'d2']=0
red_tmp_10 = red_tmp.drop_duplicates('patid')
red_tmp_10=red_tmp_10[['patid','d2']]
red_tmp_date = red_tmp[red_tmp['d2'] == 1]
red_tmp_date.loc[:,'d2_date'] = red_tmp_date['date'] - red_tmp_date['fst_dt']
red_tmp_date = red_tmp_date.drop_duplicates('patid', keep='last')
red_tmp_date = red_tmp_date[['patid','d2_date']]
red = pd.merge(red, red_tmp_10, on='patid', how ='left')
red = pd.merge(red, red_tmp_date, on = 'patid', how = 'left')

red_tmp = pd.merge(red, d5, on = 'patid', how = 'left')
red_tmp.loc[red_tmp['fst_dt']<red_tmp['date'],'d5']=1
red_tmp.loc[red_tmp['d5']!=1,'d5']=0
red_tmp_10 = red_tmp.drop_duplicates('patid')
red_tmp_10=red_tmp_10[['patid','d5']]
red_tmp_date = red_tmp[red_tmp['d5'] == 1]
red_tmp_date.loc[:,'d5_date'] = red_tmp_date['date'] - red_tmp_date['fst_dt']
red_tmp_date = red_tmp_date.drop_duplicates('patid', keep='last')
red_tmp_date = red_tmp_date[['patid','d5_date']]
red = pd.merge(red, red_tmp_10, on='patid', how ='left')
red = pd.merge(red, red_tmp_date, on = 'patid', how = 'left')

red = pd.merge(red,meta, on='patid', how='left')

C:\Users\peter\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\peter\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [65]:
red_8 = red[red['drug8_record']==1]
red_no_8 = red[red['drug8_record']==0]

In [57]:
len(red_8[red_8['d1_record']==1]), len(red_8[red_8['d1_record']==0]), len(red_no_8[red_no_8['d1_record']==1]), len(red_no_8[red_no_8['d1_record']==0])

(127, 233410, 867, 204555)

In [14]:
d = np.array([[127, 233410], [867, 204555]])
chi2_contingency(d)

(652.3070287225964,
 7.036865370774887e-144,
 1,
 array([[   528.83248322, 233008.16751678],
        [   465.16751678, 204956.83248322]]))

# Matching with binary columns

In [14]:
red_no_8_copy = copy.deepcopy(red_no_8)
red_8.index = np.arange(len(red_8))
no_index = []
yes_index = []
for i in range(len(red_8)):
    v1 = red_8.loc[i, 'drug3']
    v2 = red_8.loc[i, 'drug10']
    v3 = red_8.loc[i, 'drug8']
    v4 = red_8.loc[i, 'd2']
    v5 = red_8.loc[i, 'd5']
    v6 = red_8.loc[i, 'GDR_CD']
    v7 = red_8.loc[i, 'yrdob']
    match = red_no_8_copy[(red_no_8_copy['drug3']==v1)&
                          (red_no_8_copy['drug10']==v2)&
                          (red_no_8_copy['drug8']==v3)&
                          (red_no_8_copy['d2']==v4)&
                         (red_no_8_copy['d5']==v5)&
                         (red_no_8_copy['GDR_CD']==v6)&
                         (red_no_8_copy['yrdob']==v7)]
    if(len(match)>0):
        yes_index.append(i)
        tmp = match.index[0]
        no_index.append(tmp)
        red_no_8_copy.drop(tmp,axis=0, inplace=True)
no_drug = red_no_8.loc[no_index,:]
yes_drug = red_8.loc[yes_index,:]

In [16]:
len(no_drug[no_drug['d1_record']==1])

410

In [17]:
len(no_drug[no_drug['d1_record']==0])

94736

In [18]:
len(yes_drug[yes_drug['d1_record'] == 1])

29

In [19]:
len(yes_drug[yes_drug['d1_record'] == 0])

95117

In [2]:
d = np.array([[410, 94736], [29, 95117]])
chi2_contingency(d)

(329.68997342111004, 1.1228239172459087e-73, 1, array([[  219.5, 94926.5],
        [  219.5, 94926.5]]))

# Matching with date columns

In [17]:
red_no_8_copy = copy.deepcopy(red_no_8)
red_8.index = np.arange(len(red_8))
red_no_8_copy = copy.deepcopy(red_no_8)
no_index = []
yes_index = []
for i in range(len(red_8)):
    match = red_no_8_copy
    v1 = red_8.loc[i, 'drug3_date']
    v2 = red_8.loc[i, 'drug10_date']
    v3 = red_8.loc[i, 'drug8_date']
    v4 = red_8.loc[i, 'd2_date']
    v5 = red_8.loc[i, 'd5_date']
    
    if np.isnan(v1):
        match = match[match['drug3_date'].isna()]
    else:
        match = match[match['drug3_date'] == v1]
    if np.isnan(v2):
        match = match[match['drug10_date'].isna()]
    else:
        match = match[match['drug10_date'] == v2]
    if np.isnan(v3):
        match = match[match['drug8_date'].isna()]
    else:
        match = match[match['drug8_date'] == v3]
    if np.isnan(v4):
        match = match[match['d2_date'].isna()]
    else:
        match = match[match['d2_date'] == v4]
    if np.isnan(v5):
        match = match[match['d5_date'].isna()]
    else:
        match = match[match['d5_date'] == v5]

    v6 = red_8.loc[i, 'GDR_CD']
    v7 = red_8.loc[i, 'yrdob']
    match = match[(match['GDR_CD']==v6) & (match['yrdob']==v7)]
    if(len(match)>0):
        yes_index.append(i)
        tmp = match.index[0]
        no_index.append(tmp)
        red_no_8_copy.drop(tmp,axis=0, inplace=True)
no_drug = red_no_8.loc[no_index,:]
yes_drug = red_8.loc[yes_index,:]

In [19]:
len(no_drug[no_drug['d1_record']==1])

399

In [20]:
len(no_drug[no_drug['d1_record']==0])

91400

In [21]:
len(yes_drug[yes_drug['d1_record'] == 1])

28

In [22]:
len(yes_drug[yes_drug['d1_record'] == 0])

91771

In [23]:
d = np.array([[408, 91178], [28, 91558]])
chi2_contingency(d)

(330.237892344217, 8.530493248095933e-74, 1, array([[  218., 91368.],
        [  218., 91368.]]))